In [83]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
import os
import datetime as dt
from datetime import datetime
import bz2
import pickle
import _pickle as cPickle

%matplotlib inline

In [124]:
tickers = pd.read_csv('ndx.csv', header=None)

In [125]:
tickers = tickers[0].to_list()

In [126]:
len(tickers)

103

In [127]:
#tickers = ['FB', 'AA', 'AAPL']
stocks = download_tickers(tickers)

[*********************100%***********************]  101 of 101 completed
saved 6311 lines
saved 10089 lines
saved 8137 lines
saved 8137 lines
saved 9612 lines
saved 6736 lines
saved 3591 lines
saved 7573 lines
saved 10874 lines
saved 1971 lines
saved 1971 lines
saved 5 lines
saved 4480 lines
saved 3566 lines
saved 2794 lines
saved 9387 lines
saved 8807 lines
saved 4941 lines
saved 5 lines
saved 9491 lines
saved 5076 lines
saved 1971 lines
saved 5255 lines
saved 7090 lines
saved 4967 lines
saved 7339 lines
saved 6783 lines
saved 10593 lines
saved 4841 lines
saved 3621 lines
saved 11828 lines
saved 5412 lines
saved 1971 lines
saved 4762 lines
saved 2132 lines
saved 1971 lines
saved 8886 lines
saved 3036 lines
saved 3308 lines
saved 7119 lines
saved 3017 lines
saved 4879 lines
saved 8876 lines
saved 3420 lines
saved 1971 lines
saved 10874 lines
saved 11583 lines
saved 4376 lines
saved 10191 lines
saved 6654 lines
saved 8301 lines
saved 185 lines
saved 234 lines
saved 6587 lines
saved 7100

In [128]:
stocks

Stock ticker        AAPL                                                  \
Data type           Open        High         Low       Close   Adj Close   
Date                                                                       
1972-06-01           NaN         NaN         NaN         NaN         NaN   
1972-06-02           NaN         NaN         NaN         NaN         NaN   
1972-06-05           NaN         NaN         NaN         NaN         NaN   
1972-06-06           NaN         NaN         NaN         NaN         NaN   
1972-06-07           NaN         NaN         NaN         NaN         NaN   
...                  ...         ...         ...         ...         ...   
2020-08-31    127.580002  131.000000  126.000000  129.039993  129.039993   
2020-09-01    132.759995  134.800003  130.529999  134.179993  134.179993   
2020-09-02    137.589996  137.979996  127.000000  131.399994  131.399994   
2020-09-03    126.910004  128.839996  120.500000  120.879997  120.879997   
2020-09-04    120.070000  123.699997  110.889999  120.959999  120.959999   

Stock ticker                      AMZN                                         \
Data type          Volume         Open         High          Low        Close   
Date                                                                            
1972-06-01            NaN          NaN          NaN          NaN          NaN   
1972-06-02            NaN          NaN          NaN          NaN          NaN   
1972-06-05            NaN          NaN          NaN          NaN          NaN   
1972-06-06            NaN          NaN          NaN          NaN          NaN   
1972-06-07            NaN          NaN          NaN          NaN          NaN   
...                   ...          ...          ...          ...          ...   
2020-08-31    225702700.0  3408.989990  3495.000000  3405.000000  3450.959961   
2020-09-01    152470100.0  3489.580078  3513.870117  3467.000000  3499.120117   
2020-09-02    200119000.0  3547.000000  3552.250000  3486.689941  3531.449951   
2020-09-03    254723200.0  3485.000000  3488.409912  3303.000000  3368.000000   
2020-09-04    332607163.0  3318.000000  3381.500000  3111.129883  3294.620117   

Stock ticker  ...        FOX                                      LBTYA  \
Data type     ...        Low      Close Adj Close     Volume       Open   
Date          ...                                                         
1972-06-01    ...        NaN        NaN       NaN        NaN        NaN   
1972-06-02    ...        NaN        NaN       NaN        NaN        NaN   
1972-06-05    ...        NaN        NaN       NaN        NaN        NaN   
1972-06-06    ...        NaN        NaN       NaN        NaN        NaN   
1972-06-07    ...        NaN        NaN       NaN        NaN        NaN   
...           ...        ...        ...       ...        ...        ...   
2020-08-31    ...  27.695000  27.799999     27.57  1509100.0  23.570000   
2020-09-01    ...  27.070000  27.490000     27.49  1343800.0  23.120001   
2020-09-02    ...  27.450001  27.990000     27.99  1138300.0  23.270000   
2020-09-03    ...  27.360001  27.510000     27.51  1804100.0  23.770000   
2020-09-04    ...  27.389999  27.760000     27.76  1433014.0  23.330000   

Stock ticker                                                         
Data type          High        Low      Close  Adj Close     Volume  
Date                                                                 
1972-06-01          NaN        NaN        NaN        NaN        NaN  
1972-06-02          NaN        NaN        NaN        NaN        NaN  
1972-06-05          NaN        NaN        NaN        NaN        NaN  
1972-06-06          NaN        NaN        NaN        NaN        NaN  
1972-06-07          NaN        NaN        NaN        NaN        NaN  
...                 ...        ...        ...        ...        ...  
2020-08-31    23.830000  23.320000  23.370001  23.370001  1702400.0  
2020-09-01    23.780001  23.120001  23.270000  23

In [144]:
sns.lineplot(data=stocks.loc[:, stocks.columns.get_level_values(1).isin(['Adj Close'])].iloc[:,3:11])

ValueError: These `style` levels are missing dashes: {'PYPL', 'NFLX'}

In [112]:
def download_tickers(tickers):
    # Fetch downloaded ticker from raw_data
    downloaded_tickers = [x[0:-5] if '.pbz2' in x else '' for x in os.listdir('raw_data/')]
    downloaded_tickers.remove('')
    # Check if tickers already are downloaded
    tickers_to_download = []
    for ticker in tickers:
        if ticker in downloaded_tickers:
            # If already downloaded, check if they need updates
            if update_ticker(ticker):
                tickers_to_download.append(ticker)
        else:
            tickers_to_download.append(ticker)

    download_dump(tickers_to_download)
    
    # Read tickers from json and return
    return load_stocks(tickers)

In [113]:
def load_stocks(tickers):
    cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    index = pd.MultiIndex.from_product([tickers, cols], names=['Stock ticker', 'Data type'])
    data = pd.concat([decompress_pickle('raw_data/' + x + '.pbz2') for x in tickers], axis=1, sort=True)
    data.columns = index
    try:
        data.index = pd.to_datetime(data.index, unit='ms')
    except ValueError:
        data.index = pd.to_datetime(data.index)
    return data

In [114]:
def update_ticker(ticker):
    last_bday = last_weekday()
    newest_date = decompress_pickle('raw_data/' + ticker + '.pbz2').index[-1]
    if newest_date == float:
        newest_date = dt.datetime.fromtimestamp(newest_date/1000).date()
    if newest_date != last_bday:
        return True 
    else:
        return False

In [115]:
def download_dump(tickers):
    if tickers != []:
        data = yf.download(tickers, period='100y', group_by='tickers')
        if len(tickers) > 1:
            for ticker in tickers:
                    clean_data = clean_df(data[ticker])
                    #clean_data.to_json('raw_data/' + ticker + '.json', date_format='epoch', date_unit='ms')
                    compressed_pickle(ticker, data[ticker])
        else:
            clean_data = clean_df(data)
            #clean_data.to_json('raw_data/' + tickers[0] + '.json', date_format='epoch', date_unit='ms')
            compressed_pickle(ticker, data)

In [116]:
def last_weekday():
    todays_day = dt.date.today().day
    if todays_day in [5, 6]:
        return dt.date.today() - dt.timedelta(days=todays_day-4)
    else:
        return dt.date.today()


In [117]:
def clean_df(df):
    before = len(df)
    df = df.dropna(axis = 0, how='all')
    print('saved', before-len(df), 'lines')
    return df

In [118]:
def compressed_pickle(ticker, data):
    with bz2.BZ2File('raw_data/' + ticker + '.pbz2', 'w') as f: 
        cPickle.dump(data, f)

In [119]:
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [120]:
test = decompress_pickle('raw_data/AAPL.pbz2')

In [121]:
test

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.101261,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095978,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088934,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.091135,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093777,73449600
...,...,...,...,...,...,...
2020-08-31,127.580002,131.000000,126.000000,129.039993,129.039993,225702700
2020-09-01,132.759995,134.800003,130.529999,134.179993,134.179993,152470100
2020-09-02,137.589996,137.979996,127.000000,131.399994,131.399994,200119000
